<a href="https://colab.research.google.com/github/kenanmorani/Audio_Signal_Processing_For_Music_Applications_idu/blob/main/Pitch%20Exctraction/Copy_of_HW3_Audio_Signal_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW3 Audio Signal Processing for Music Applications course
Izmir Democracy University, Electrical and Electronics Program    
Submitted to Professor Barsi Bozkurt      
Submitted by Kenan MORANI     

# Installing and importing the required libraries

In [ ]:
!pip install essentia mir_eval pysoundfile

     |████████████████████████████████| 12.0MB 8.7MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
  Created wheel for mir-eval: filename=mir_eval-0.6-cp36-none-any.whl size=96515 sha256=5c10d91a64e9a1437e9de7457000544f3b30fd94985d81e8ee7ab04eeec9ef1d
  Stored in directory: /root/.cache/pip/wheels/49/ce/30/730fa72addf275e49d90683b01b3613048b4be3bf7ff8eb6ec
Successfully built mir-eval


In [ ]:
import os
from essentia import *
from essentia.standard import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import get_window
import soundfile as sf
import urllib.request
from IPython.display import Audio
import mir_eval

In [ ]:
#Downloading a standard mir dataset
!pip install mirdata
import mirdata
orchset = mirdata.initialize('orchset')
orchset.download()

     |████████████████████████████████| 6.1MB 6.6MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 5.6MB 32.0MB/s 
     |████████████████████████████████| 184kB 47.6MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
  Created wheel for jams: filename=jams-0.3.4-cp36-none-any.whl size=64925 sha256=0c64d5cbf943a772730de7d339795b45dee137ebbd96fbee43bffae85719c94b
  Stored in directory: /root/.cache/pip/wheels/2b/b8/c7/092096a3af0eb864cca4e79a44bc883dee5b9767da2c6f8ab3
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp36-none-any.whl size=5591954 sha256=4b8350995ef1089402f037355975429f28201150a47c70c237ca7611ea0aca71
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
  Created wheel for librosa: filename=librosa-0.8.0-cp36-none-any.whl size=201375 sha256=e20c

INFO: Downloading ['all'] to /root/mir_datasets/orchset
INFO: [all] downloading Orchset_dataset_0.zip
311MB [01:09, 4.70MB/s]                           


# Exctracting f0s

In [ ]:
#Collecting audio file names 
all_files = []
for root, dirs, files in os.walk('/root/mir_datasets/orchset/audio/mono/'):
    for file in files:
        if file.endswith('.wav'):
            file_name = os.path.join(root,file)
            all_files.append(file_name)

In [ ]:
for file in all_files:
 x, fs = sf.read(file)
 x = x / np.max(np.abs(x))#amplitude normalisation
 minF0 = 50 #in Hz
 maxF0 = 2000 #in Hz
 windowSize = 4096
 hopSize = 1024
 w = get_window('blackman', windowSize)
 startIndexes = np.arange(0, x.size - windowSize, hopSize, dtype = int)
 numWindows = startIndexes.size
 Audio(x, rate=fs)
 run_predominant_melody = PitchMelodia(guessUnvoiced = True,
                                      frameSize = windowSize,
                                      hopSize = hopSize,
                                      maxFrequency = maxF0,
                                      minFrequency = minF0,
                                      sampleRate = fs);
 f0, confidence = run_predominant_melody(x)


# Using essentia-melody algorithm

In [ ]:
#Collecting mel file names for Annotations 
all_midi = []
for root, dirs, files in os.walk('/root/mir_datasets/orchset/GT'):
    for file in files:
        if file.endswith('.mel'):
            file_name = os.path.join(root,file)
            all_midi.append(file_name)
for file in all_midi:        
   #Reading ground-truth
   ref_time, ref_freq = mir_eval.io.load_time_series(file)

In [ ]:
#Saving estimated values to file
for file in all_files:
 fileNameF0 = file.replace('.wav','.f0_preDomMel.txt')
 startIndexes = np.arange(0, x.size - windowSize, hopSize, dtype = int)
 with open(fileNameF0, 'w') as f:
    for k in range(startIndexes.size):
        time = (startIndexes[k] + windowSize / 2) / fs
        f.write(str(time) + '\t' + str(f0[k]) + '\n')

NameError: ignored

In [ ]:
#Collecting text file names 
all_text = []
for root, dirs, files in os.walk('/root/mir_datasets/orchset/audio/mono/'):
    for file in files:
        if file.endswith('.txt'):
         
            est_time, est_freq = mir_eval.io
            scores = mir_eval.melody.evaluate(ref_time, ref_freq,est_time, est_freq)
            for score in scores:
              print(score,'\t',scores[score])

IndentationError: ignored

In [ ]:
for file in all_files:
 x, fs = sf.read(file)
 x = x / np.max(np.abs(x))#amplitude normalisation
 minF0 = 50 #in Hz
 maxF0 = 2000 #in Hz
 windowSize = 4096
 hopSize = 1024
 w = get_window('blackman', windowSize)
 startIndexes = np.arange(0, x.size - windowSize, hopSize, dtype = int)
 numWindows = startIndexes.size
 Audio(x, rate=fs)
 run_predominant_melody = PitchMelodia(guessUnvoiced = True,
                                      frameSize = windowSize,
                                      hopSize = hopSize,
                                      maxFrequency = maxF0,
                                      minFrequency = minF0,
                                      sampleRate = fs);
 f0, confidence = run_predominant_melody(x)
 est_time, est_freq = mir_eval.io.load_wav(x)
 scores = mir_eval.melody.evaluate(ref_time, ref_freq,est_time, est_freq)
  for score in scores:
    print(score,'\t',scores[score])



Voicing Recall 	 0.9198641765704584
Voicing False Alarm 	 0.2647058823529412
Raw Pitch Accuracy 	 0.0023769100169779285
Raw Chroma Accuracy 	 0.04380305602716469
Overall Accuracy 	 0.010741859684457872
Voicing Recall 	 0.9198641765704584
Voicing False Alarm 	 0.2647058823529412
Raw Pitch Accuracy 	 0.0023769100169779285
Raw Chroma Accuracy 	 0.04380305602716469
Overall Accuracy 	 0.010741859684457872
Voicing Recall 	 0.9198641765704584
Voicing False Alarm 	 0.2647058823529412
Raw Pitch Accuracy 	 0.0023769100169779285
Raw Chroma Accuracy 	 0.04380305602716469
Overall Accuracy 	 0.010741859684457872
Voicing Recall 	 0.9198641765704584
Voicing False Alarm 	 0.2647058823529412
Raw Pitch Accuracy 	 0.0023769100169779285
Raw Chroma Accuracy 	 0.04380305602716469
Overall Accuracy 	 0.010741859684457872
Voicing Recall 	 0.9198641765704584
Voicing False Alarm 	 0.2647058823529412
Raw Pitch Accuracy 	 0.0023769100169779285
Raw Chroma Accuracy 	 0.04380305602716469
Overall Accuracy 	 0.010741859

# Using crepe algorithm

In [ ]:
!pip install crepe
import crepe

In [ ]:
for file in all_files[:10]:
  #Running Essentia predominant melody extraction and saving to an output file
    hopSize = 441#10 miliseconds
    windowSize = 4096
    fs = 44100
    x, fs = sf.read(file)
    x = x / np.max(np.abs(x))#amplitude normalisation
    #x = EqualLoudness()(x)
    est_time, est_freq, confidence, activation = crepe.predict(x, fs,viterbi=True)
    #Running evaluation and ptinting
    scores = mir_eval.melody.evaluate(ref_time, ref_freq,est_time, est_freq)
    for score in scores:
      print(score,'\t',scores[score])

33/33 [==============================] - 51s 2s/step
Voicing Recall 	 0.999660441426146
Voicing False Alarm 	 1.0
Raw Pitch Accuracy 	 0.014261460101867572
Raw Chroma Accuracy 	 0.027164685908319185
Overall Accuracy 	 0.014098690835850957
77/77 [==============================] - 121s 2s/step
Voicing Recall 	 0.999660441426146
Voicing False Alarm 	 1.0
Raw Pitch Accuracy 	 0.0
Raw Chroma Accuracy 	 0.03531409168081494
Overall Accuracy 	 0.0
74/74 [==============================] - 119s 2s/step
Voicing Recall 	 0.999660441426146
Voicing False Alarm 	 1.0
Raw Pitch Accuracy 	 0.00033955857385398983
Raw Chroma Accuracy 	 0.061799660441426145
Overall Accuracy 	 0.0003356831151393085
67/67 [==============================] - 105s 2s/step
Voicing Recall 	 0.999660441426146
Voicing False Alarm 	 1.0
Raw Pitch Accuracy 	 0.0
Raw Chroma Accuracy 	 0.014940577249575551
Overall Accuracy 	 0.0
49/93 [==============>...............] - ETA: 1:09

KeyboardInterrupt: ignored